In [27]:
import pandas as pd
import numpy as np

# 加载清洗后数据
df = pd.read_excel(r'E:\shenzhen_house_price_project\data\clean\清洗后数据.xlsx')
print(f"✅ 数据加载成功，共 {len(df):,} 条记录")
df.head(3)

 # 3.1 区域价值全景：哪里贵？哪里值得买？
"""
SQL 模拟：
SELECT
    district,
    AVG(per_price) AS avg_price,
    AVG(subway) AS subway_ratio,
    AVG(school) AS school_ratio,
    COUNT(*) AS listing_count
FROM housing
GROUP BY district;
"""

region_summary = (
    df.groupby('district')
    .agg(
        avg_price=('per_price', 'mean'),
        subway_ratio=('subway', 'mean'),
        school_ratio=('school', 'mean'),
        listing_count=('per_price', 'size')
    )
    .round(4)
    .reset_index()
)

# 转为百分比（Tableau 可直接用小数，此处便于阅读）
region_summary[['subway_ratio', 'school_ratio']] *= 100
region_summary = region_summary.round(1)

print("=== 3.1 区域价值全景 ===")
print(region_summary.sort_values('avg_price', ascending=False))

✅ 数据加载成功，共 29,904 条记录
=== 3.1 区域价值全景 ===
  district  avg_price  subway_ratio  school_ratio  listing_count
1       南山       10.8          94.7          47.7           2904
6       福田        8.9          97.0          70.3           3680
4       宝安        5.9          84.2          42.7           3790
8       龙华        5.3          76.7          45.5           3498
7       罗湖        5.1          99.4          60.1           4159
5       盐田        4.1          96.8          43.0           2723
0       光明        4.0          73.9          24.5           2041
9       龙岗        3.9          87.5          63.0           4127
2       坪山        2.9          73.7          39.2           2761
3     大鹏新区        2.8           0.0          35.3            221


In [28]:
# 3.2 客群画像：不同预算人群在买什么？
# 定义客群
def get_segment(price):
    if price < 500:
        return '刚需'
    elif price <= 1000:
        return '改善'
    else:
        return '高端'

df['customer_segment'] = df['total_price'].apply(get_segment)

"""
SQL 模拟（客群 × 卧室数）：
SELECT
    CASE
        WHEN total_price < 500 THEN '刚需'
        WHEN total_price BETWEEN 500 AND 1000 THEN '改善'
        ELSE '高端'
    END AS segment,
    roomnum,
    COUNT(*) AS count
FROM housing
WHERE roomnum IS NOT NULL
GROUP BY segment, roomnum;
"""

segment_room = (
    df.groupby(['customer_segment', 'roomnum'])
    .size()
    .unstack(fill_value=0)
    .loc[['刚需', '改善', '高端']]  # 固定顺序
)
print("\n=== 3.2 客群 × 卧室数分布 ===")
print(segment_room)

"""
SQL 模拟（客群 × 行政区）：
SELECT
    segment,
    district,
    COUNT(*) AS count
FROM (
    SELECT *,
           CASE ... END AS segment
    FROM housing
) t
GROUP BY segment, district;
"""

segment_district = (
    df.groupby(['customer_segment', 'district'])
    .size()
    .unstack(fill_value=0)
    .loc[['刚需', '改善', '高端']]
)
print("\n=== 客群 × 行政区热力表 ===")
print(segment_district)


=== 3.2 客群 × 卧室数分布 ===
roomnum              1     2     3     4     5    6    7   8   9
customer_segment                                                
刚需                2080  4854  8968  2139   161   15    2   0   0
改善                  51   712  3293  2250   649   87   19  10   2
高端                   8   117  1050  1947  1031  273  103  53  30

=== 客群 × 行政区热力表 ===
district            光明    南山    坪山  大鹏新区    宝安    盐田    福田    罗湖    龙华    龙岗
customer_segment                                                            
刚需                1625   415  2669   201  2025  1919   793  3185  2060  3327
改善                 399   788    83    13  1025   613  1497   782  1175   698
高端                  17  1701     9     7   740   191  1390   192   263   102


In [29]:
# === 3.3 核心属性影响：学区和地铁溢价分析 ===

def calc_premium(price_with, price_without):
    """
    计算属性溢价百分比
    :param price_with: 有属性的平均单价
    :param price_without: 无属性的平均单价
    :return: 溢价百分比（保留1位小数）
    """
    if price_without == 0 or pd.isna(price_without):
        return np.nan
    premium = (price_with - price_without) / price_without * 100
    return round(premium, 1)

# --- 全局溢价 ---
avg_price_school_yes = df[df['school'] == 1]['per_price'].mean()
avg_price_school_no  = df[df['school'] == 0]['per_price'].mean()
school_premium_pct = calc_premium(avg_price_school_yes, avg_price_school_no)

avg_price_subway_yes = df[df['subway'] == 1]['per_price'].mean()
avg_price_subway_no  = df[df['subway'] == 0]['per_price'].mean()
subway_premium_pct = calc_premium(avg_price_subway_yes, avg_price_subway_no)

print("\n=== 3.3 属性溢价（全局） ===")
print(f"学区溢价: +{school_premium_pct}%")
print(f"地铁溢价: +{subway_premium_pct}%")

# --- 分区域学区溢价 ---
school_by_district = df.pivot_table(
    values='per_price',
    index='district',
    columns='school',
    aggfunc='mean'
).round(2)

# 确保列顺序：0 -> non_school, 1 -> school
school_by_district.columns = ['non_school', 'school']

# 计算溢价百分比（处理除零或 NaN）
school_by_district['premium_%'] = (
    (school_by_district['school'] - school_by_district['non_school'])
    / school_by_district['non_school'] * 100
).round(1)

# 按溢价降序排序，取 Top 5
top_premium_districts = school_by_district.sort_values('premium_%', ascending=False).head()

print("\n=== 分区域学区溢价（Top 5） ===")
print(top_premium_districts[['non_school', 'school', 'premium_%']])

# 构造全局溢价 DataFrame（便于可视化）
global_premium = pd.DataFrame({
    'factor': ['school', 'subway'],
    'premium_%': [school_premium_pct, subway_premium_pct]
})
global_premium.to_csv(r'E:\shenzhen_house_price_project\data\clean\global_premium.csv', index=False, encoding='utf-8-sig')


=== 3.3 属性溢价（全局） ===
学区溢价: +6.3%
地铁溢价: +46.8%

=== 分区域学区溢价（Top 5） ===
          non_school  school  premium_%
district                               
龙华              4.94    5.72       15.8
大鹏新区            2.74    3.03       10.6
宝安              5.72    6.14        7.3
龙岗              3.73    3.99        7.0
罗湖              4.93    5.20        5.5


In [30]:
# 3.4 楼层偏好分析
# 假设 C_floor 值为：'low', 'middle', 'high'
"""
SQL 模拟：
SELECT
    C_floor,
    COUNT(*) * 100.0 / (SELECT COUNT(*) FROM housing) AS share_pct,
    AVG(per_price) AS avg_price
FROM housing
GROUP BY C_floor;
"""

floor_analysis = (
    df.groupby('C_floor')
    .agg(
        count=('per_price', 'size'),
        avg_price=('per_price', 'mean')
    )
    .round(2)
)
floor_analysis['share_%'] = (floor_analysis['count'] / floor_analysis['count'].sum() * 100).round(1)
floor_analysis = floor_analysis[['share_%', 'avg_price']]

print("\n=== 3.4 楼层偏好 ===")
print(floor_analysis)

# 计算相对溢价（以 low 为基准）
if 'low' in floor_analysis.index:
    base_price = floor_analysis.loc['low', 'avg_price']
    floor_analysis['vs_low_%'] = ((floor_analysis['avg_price'] - base_price) / base_price * 100).round(1)
    print("\nvs 低层溢价：")
    print(floor_analysis['vs_low_%'])


=== 3.4 楼层偏好 ===
         share_%  avg_price
C_floor                    
high        34.1       5.77
low         29.9       5.89
middle      36.0       5.47

vs 低层溢价：
C_floor
high     -2.0
low       0.0
middle   -7.1
Name: vs_low_%, dtype: float64


In [33]:
import os

# 定义输出目录（相对于当前 notebook）
output_dir = r'E:\shenzhen_house_price_project\data\clean'
os.makedirs(output_dir, exist_ok=True)  # 自动创建目录

print("正在导出 Tableau 友好格式的数据...")

# 1. 区域价值汇总表（用于柱状图、散点图）
region_summary_for_tableau = region_summary.copy()
region_summary_for_tableau.columns = ['行政区', '平均单价_万元_per_m2', '地铁房占比_%', '学区房占比_%', '房源数量']
region_summary_for_tableau.to_csv(os.path.join(output_dir, 'region_summary.csv'), index=False, encoding='utf-8-sig')

# 2. 客群 × 卧室数分布（转为长格式，适合堆叠柱状图）
# 确保使用长格式，并且包含正确的列名
segment_room_long = (
    segment_room
    .reset_index()
    .melt(
        id_vars='customer_segment',
        var_name='卧室数量',
        value_name='房源数量'
    )
    .rename(columns={'customer_segment': '客户类型'})
)

# 确保数据类型正常
segment_room_long['卧室数量'] = segment_room_long['卧室数量'].astype(int)

# ✅ 关键：确保 header=True，encoding='utf-8-sig'（Windows 兼容）


segment_room_long.to_csv(
    os.path.join(output_dir, 'segment_room.csv'),
    index=False,
    encoding='utf-8-sig',  # ✅ 必须！避免中文乱码
    header=True           # ✅ 明确指定有表头
)

# 3. 客群 × 行政区热力表（同样转为长格式）
segment_district_long = segment_district.reset_index().melt(
    id_vars='customer_segment',
    var_name='行政区',
    value_name='房源数量'
)
segment_district_long.rename(columns={'customer_segment': '客户类型'}, inplace=True)
segment_district_long.to_csv(os.path.join(output_dir, 'segment_district.csv'), index=False, encoding='utf-8-sig')

# 4. 楼层偏好分析
floor_analysis_for_tableau = floor_analysis.reset_index()
floor_analysis_for_tableau.rename(columns={
    'C_floor': '楼层类型',
    'share_%': '市场份额_%',
    'avg_price': '平均单价_万元_per_m2'
}, inplace=True)
floor_analysis_for_tableau.to_csv(os.path.join(output_dir, 'floor_analysis.csv'), index=False, encoding='utf-8-sig')

# 5. 全局属性溢价（用于分组柱状图）
global_premium_for_tableau = global_premium.copy()
global_premium_for_tableau['factor'] = global_premium_for_tableau['factor'].map({
    'school': '学区房',
    'subway': '地铁房'
})
global_premium_for_tableau.rename(columns={
    'factor': '属性类型',
    'premium_%': '溢价百分比_%'
}, inplace=True)
global_premium_for_tableau.to_csv(os.path.join(output_dir, 'global_premium.csv'), index=False, encoding='utf-8-sig')

# 6. 带客群标签的明细数据（用于 Tableau 交互式仪表盘）
df_for_tableau = df.copy()
df_for_tableau.rename(columns={
    'district': '行政区',
    'per_price': '单价_万元_per_m2',
    'total_price': '总价_万元',
    'roomnum': '卧室数量',
    'hall': '厅数量',
    'school': '是否学区房',
    'subway': '是否近地铁',
    'C_floor': '楼层类型',
    'customer_segment': '客户类型'
}, inplace=True)

# 确保二值字段为整数（Tableau 显示更清晰）
df_for_tableau['是否学区房'] = df_for_tableau['是否学区房'].astype(int)
df_for_tableau['是否近地铁'] = df_for_tableau['是否近地铁'].astype(int)

df_for_tableau.to_csv(os.path.join(output_dir, 'tableau_input.csv'), index=False, encoding='utf-8-sig')

print("✅ 所有 Tableau 可用数据已成功导出至:", output_dir)


正在导出 Tableau 友好格式的数据...
✅ 所有 Tableau 可用数据已成功导出至: E:\shenzhen_house_price_project\data\clean
